# A Tutorial on Model Training in Python with Keras

In [1]:
# cell 1: 永久性路径配置
import sys
from pathlib import Path

def add_project_root():
    """add python project root to sys.path"""
    notebook_path = Path().absolute()  # Get the absolute path of the notebook
    project_root = notebook_path.parent  # Go up two levels to the project root

    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))  # search project root first
        print(f"✅ project root added: {project_root}")
    else:
        print("⏩ project root already exists")
    
    return project_root

project_root = add_project_root()


✅ project root added: e:\Data Files\Projects\Embedded Systems\f103_demo_vscode


In [2]:
from data_processing import create_windows
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from model import MyCNNModel
import keras


## 1. Pre-process data

In [3]:
import yaml
with open((project_root / 'config.yaml'), 'r') as file:
    config = yaml.safe_load(file)
# Load data
ds_config = config['datasets'][config['active_dataset']]
print(f"Loading dataset from {ds_config['path']}")
data = pd.read_csv(project_root / ds_config['path'])
data_columns = ds_config['features']
norm_windows, labels, _ = create_windows(
    data,
    window_length=config['data_processing']['window']['length'],
    step_size=config['data_processing']['window']['step_size'],
    overlap=None,
    feature_columns=data_columns,
    label_column=ds_config['label_column'],
)
print(norm_windows[0])
print(norm_windows.shape)

Loading dataset from ./data/mafaulda19.csv
[[0.1268389  0.50210248 0.42535302 0.49732183 0.45425745 0.51582176
  0.34823428]
 [0.12390014 0.79841099 0.56313183 0.57897221 0.47493213 0.53154488
  0.36121152]
 [0.13653318 0.38906528 0.36739579 0.56287525 0.45272538 0.52438231
  0.32922548]
 [0.11404636 0.76992723 0.61350561 0.61937753 0.48076297 0.53406649
  0.34672639]
 [0.14071314 0.49854315 0.33360714 0.56348694 0.46513774 0.55279058
  0.32424605]
 [0.13252692 0.65480122 0.58415628 0.52554504 0.47726446 0.59921238
  0.3411011 ]
 [0.11675141 0.64901484 0.39020151 0.42395862 0.46699559 0.58795604
  0.34841398]
 [0.12332849 0.53862245 0.50149669 0.36988451 0.46345459 0.59455294
  0.36118287]
 [0.12170199 0.77000319 0.48721797 0.34031024 0.46977881 0.59797623
  0.38057138]
 [0.13794176 0.44185517 0.39682818 0.30800992 0.45284342 0.59034973
  0.37364967]
 [0.13480719 0.81156668 0.55976619 0.38185251 0.47349449 0.60342638
  0.40327999]
 [0.13193159 0.47252951 0.31846178 0.40229121 0.4571634

## 2. split train, validation and test data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    norm_windows, labels, test_size=0.2, random_state=42
)
y_train = keras.utils.to_categorical(y_train, num_classes=len(np.unique(labels)))
y_test = keras.utils.to_categorical(y_test, num_classes=len(np.unique(labels)))

print(X_train.shape, y_train.shape)
model_input_shape = (X_train.shape[1], X_train.shape[2]) # (window_length, num_features)
num_classes = y_train.shape[1]  # Number of classes for one-hot encoding
print(y_train)
print(num_classes)

(71898, 128, 7) (71898, 6)
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
6


In [5]:
# Model initialization
tf_model = MyCNNModel(input_shape=model_input_shape,
                      num_classes=num_classes)
# Model training
training_config = config['model']['training']
evaluation_config = config['model']['evaluation']
# tf_model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=training_config['learning_rate']),
#     loss=training_config['loss'],
#     metrics=training_config['metrics'],
#     # learning_rate=training_config['learning_rate'],
#     # batch_size=training_config['batch_size'],
# )

# tf_model.fit(
#     X_train,
#     y_train,
#     validation_split=training_config['validation_split'],
#     epochs=training_config['epochs'],
#     batch_size=training_config['batch_size'],
#     callbacks=[
#         keras.callbacks.EarlyStopping(
#             monitor='val_loss',
#             patience=training_config['early_stopping_patience'],
#             restore_best_weights=True
#         ),
#         keras.callbacks.ModelCheckpoint(
#             filepath=str(project_root / 'model.h5'),
#             save_best_only=True,
#             monitor='val_loss'
#         )
#     ]
# )

tf_model.compile_model(learning_rate=training_config['learning_rate'],
                       loss=training_config['loss'],
                       metrics=training_config['metrics'])
history = tf_model.train(
                        X_train,
                        y_train,
                        validation_split=training_config['validation_split'],
                        epochs=training_config['epochs'],
                        batch_size=training_config['batch_size'],
                        callbacks=[
                            keras.callbacks.EarlyStopping(
                                monitor='val_loss',
                                patience=training_config['early_stopping_patience'],
                                restore_best_weights=True
                            ),
                            keras.callbacks.ModelCheckpoint(
                                filepath=str(project_root / 'model.h5'),
                                save_best_only=True,
                                monitor='val_loss'
                            )
                        ]
                        )
print(history)

c:\.venv\EdgeAI\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.2389 - auc: 0.5823 - f1_score: 0.2004 - loss: 1.7542 - precision: 0.3039 - recall: 0.0029  

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.2464 - auc: 0.5892 - f1_score: 0.2089 - loss: 1.7468 - precision: 0.3443 - recall: 0.0044 - val_accuracy: 0.3770 - val_auc: 0.7581 - val_f1_score: 0.3159 - val_loss: 1.4907 - val_precision: 0.8769 - val_recall: 0.0728
Epoch 2/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.4487 - auc: 0.8019 - f1_score: 0.4189 - loss: 1.3926 - precision: 0.8487 - recall: 0.1276

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.4531 - auc: 0.8052 - f1_score: 0.4244 - loss: 1.3841 - precision: 0.8524 - recall: 0.1309 - val_accuracy: 0.5974 - val_auc: 0.8977 - val_f1_score: 0.5988 - val_loss: 1.0919 - val_precision: 0.9240 - val_recall: 0.2469
Epoch 3/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.6183 - auc: 0.9092 - f1_score: 0.6209 - loss: 1.0252 - precision: 0.9140 - recall: 0.3171

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.6207 - auc: 0.9102 - f1_score: 0.6233 - loss: 1.0193 - precision: 0.9136 - recall: 0.3217 - val_accuracy: 0.7067 - val_auc: 0.9406 - val_f1_score: 0.7146 - val_loss: 0.8291 - val_precision: 0.9170 - val_recall: 0.4427
Epoch 4/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.7086 - auc: 0.9438 - f1_score: 0.7131 - loss: 0.8001 - precision: 0.9037 - recall: 0.4772

...

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.9919 - auc: 0.9997 - f1_score: 0.9919 - loss: 0.0467 - precision: 0.9927 - recall: 0.9908 - val_accuracy: 0.9926 - val_auc: 0.9998 - val_f1_score: 0.9925 - val_loss: 0.0424 - val_precision: 0.9934 - val_recall: 0.9914
Epoch 45/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.9939 - auc: 0.9998 - f1_score: 0.9939 - loss: 0.0408 - precision: 0.9946 - recall: 0.9928 - val_accuracy: 0.9909 - val_auc: 0.9998 - val_f1_score: 0.9907 - val_loss: 0.0444 - val_precision: 0.9917 - val_recall: 0.9892
Epoch 46/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.9931 - auc: 0.9998 - f1_score: 0.9932 - loss: 0.0428 - precision: 0.9940 - recall: 0.9921

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.9932 - auc: 0.9998 - f1_score: 0.9932 - loss: 0.0427 - precision: 0.9941 - recall: 0.9922 - val_accuracy: 0.9933 - val_auc: 0.9998 - val_f1_score: 0.9932 - val_loss: 0.0414 - val_precision: 0.9942 - val_recall: 0.9928
Epoch 47/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.9942 - auc: 0.9998 - f1_score: 0.9942 - loss: 0.0396 - precision: 0.9949 - recall: 0.9935

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9942 - auc: 0.9998 - f1_score: 0.9942 - loss: 0.0397 - precision: 0.9948 - recall: 0.9935 - val_accuracy: 0.9952 - val_auc: 0.9999 - val_f1_score: 0.9951 - val_loss: 0.0364 - val_precision: 0.9955 - val_recall: 0.9942
Epoch 48/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.9933 - auc: 0.9997 - f1_score: 0.9933 - loss: 0.0402 - precision: 0.9941 - recall: 0.9927

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9934 - auc: 0.9997 - f1_score: 0.9934 - loss: 0.0401 - precision: 0.9941 - recall: 0.9927 - val_accuracy: 0.9944 - val_auc: 0.9999 - val_f1_score: 0.9943 - val_loss: 0.0348 - val_precision: 0.9948 - val_recall: 0.9932
Epoch 49/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9948 - auc: 0.9998 - f1_score: 0.9948 - loss: 0.0357 - precision: 0.9956 - recall: 0.9942 - val_accuracy: 0.9941 - val_auc: 0.9999 - val_f1_score: 0.9940 - val_loss: 0.0363 - val_precision: 0.9945 - val_recall: 0.9933
Epoch 50/50
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9953 - auc: 0.9998 - f1_score: 0.9953 - loss: 0.0350 - precision: 0.9958 - recall: 0.9949

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9954 - auc: 0.9998 - f1_score: 0.9954 - loss: 0.0348 - precision: 0.9958 - recall: 0.9950 - val_accuracy: 0.9944 - val_auc: 0.9999 - val_f1_score: 0.9943 - val_loss: 0.0337 - val_precision: 0.9950 - val_recall: 0.9935


## convert to tensorflow lite

In [12]:
import tensorflow as tf
# evaluate the model
history = tf_model.evaluate(X_test, y_test)
print(history)

# Save the model as SavedModel format
tf_model.export('saved_model_dir', save_format='tf')

# Convert to TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model_dir')
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(X_test).batch(1).take(100):
        yield [tf.cast(input_value, tf.float32)]

# Optional: Provide a representative dataset to enable full integer quantization (better for STM32)
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save as .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model is saved as: model.tflite")



562/562 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9959 - auc: 0.9998 - f1_score: 0.9959 - loss: 0.0350 - precision: 0.9967 - recall: 0.9956
[0.03541061282157898, 0.995326817035675, 0.9960452318191528, 0.9948261380195618, <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([0.99449164, 0.99864036, 0.99284315, 0.9994994 , 0.9903515 ,
       0.99639106], dtype=float32)>, 0.999830961227417]
INFO:tensorflow:Assets written to: saved_model_dir\assets


INFO:tensorflow:Assets written to: saved_model_dir\assets


Saved artifact at 'saved_model_dir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 7), dtype=tf.float32, name=None)
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2406968851296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406968850768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406968851472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406981071120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406968845840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406981067072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406981068480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2406981071824: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model is saved as: model.tflite


### record quantization parameters

In [14]:

# load the TFLite model and print input/output quantization parameters
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input quantization parameters
print("Input quantization params:")
print(" scale:", input_details[0]['quantization'][0])
print(" zero_point:", input_details[0]['quantization'][1])

# Print output quantization parameters
print("Output quantization params:")
print(" scale:", output_details[0]['quantization'][0])
print(" zero_point:", output_details[0]['quantization'][1])

# write the quantization to yaml file
quantization_params = {
    'input_scale': input_details[0]['quantization'][0],
    'input_zero_point': input_details[0]['quantization'][1],
    'output_scale': output_details[0]['quantization'][0],
    'output_zero_point': output_details[0]['quantization'][1]
}
with open('quantization_params.yaml', 'w') as f:
    yaml.dump(quantization_params, f)
print("Quantization parameters are saved as: quantization_params.yaml")


Input quantization params:
 scale: 0.003921568859368563
 zero_point: -128
Output quantization params:
 scale: 0.00390625
 zero_point: -128
Quantization parameters are saved as: quantization_params.yaml


## test model

In [28]:
from keras.utils import custom_object_scope

with custom_object_scope({'MyCNNModel': MyCNNModel}):
    model_test = keras.models.load_model(project_root / 'model.h5')
# Model evaluation
results = model_test.evaluate(X_test, y_test, batch_size=32)
print(f"Test loss: {results[0]}")
for name, value in zip(model_test.metrics_names, results):
    print(f"{name}: {value}")

TypeError: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class 'model.MyCNNModel'>, which is the default behavior. You can override this default behavior by defining a `from_config(cls, config)` class method to specify how to create an instance of MyCNNModel from its config.

Received config={'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}

Error encountered during deserialization: MyCNNModel.__init__() got an unexpected keyword argument 'trainable'